<a href="https://colab.research.google.com/github/PurpleDin0/698S_BOG/blob/master/698S_Project_BOG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraper Execution Notebook
BLUF: THis notebook imports the relevant python scraping scripts from github, builds the envirnoment, executes the scripts, saves the data locally, performs basic NLP on the recieved data, then prompts the user to save the resulting information/data (either locally, or to Google Drive).

* [ ] The websites to be searched are located here [Link to website list](https://drive.google.com/file/d/1y8FL1rBu8yb2HEn7-ok-ILy1Wq3UrDZU/view?usp=sharing).  Note: website list restricted, for access see the author.
* [ ] The scrapped data should be searched for the terms included in this file [Link to search terms list](https://drive.google.com/file/d/18MyoEiAYTG6w6JRejv5d4qVK4RxYyRu2/view?usp=sharing).  Note: file access restricted, for access see the author.

Problem Statement:
Builds a tool to scrape websites and then analyze the information for key terms
and provide the analyst with a rank-ordered list of documents that require human review. 
1. [ ] Construct a set of Python scripts that can be used to scrape a set of [3 websites](https://drive.google.com/file/d/1y8FL1rBu8yb2HEn7-ok-ILy1Wq3UrDZU/view?usp=sharing).  
  - [ ] 1.1. Build script to scrape website 1.  
  - [ ] 1.2. Build script to scrape website 2.  
  - [ ] 1.3. Build script to scrape website 3.  
2. [ ] Construct a python Notebook that executes the python script and then performs natural language processing (NLP) to the scrapped data.  
  - [ ] 2.1. Build the initial section that executes the python scripts.
  - [ ] 2.2. Rank order the scrapped results by relevance to the analyist. The ranking will be based on the [list of key terms](https://drive.google.com/file/d/18MyoEiAYTG6w6JRejv5d4qVK4RxYyRu2/view?usp=sharing).
  - [ ] 2.3. include an ability to automate the translation of selected
portions of the scraped content.
  - [ ] 2.4. Provide the user with the ability to save the scrapped content.

- Other possible things: 
  -  The code should allow the list of keywords to change over time.  
  -  The system  should immediately flag certain keywords for human review regardless of other content in the document.  
  -  The system should be able to retrieve and analyze office documents (PDFs,
spreadsheets, and word processor documents). 
  - The system should perform sentiment analysis of selected content.


In [0]:
!cp /content/698S_BOG/websites.csv '/content/drive/My Drive/Colab Notebooks/Coursework/698S/Project'

# Prepare the environment 
0. git the repo and install all dependancies
1. Install any dependancies
2. Load the list of search sites and search terms

In [1]:
import os
## 0. Clone the repo and then change the working directory to the cloned folder
git_repo =  '698S_BOG'  # Repo we want to clone
git_user = 'PurpleDin0' # User/org we want to clone the repo from
!git clone https://github.com/{git_user}/{git_repo}.git
repo_dir = os.path.join(os.getcwd(), git_repo)
%cd {repo_dir}

## 1. Install the required libraries
!pip install -r requirements.txt

# Caution: Once complete make sure to restart the runtime if required. 

Cloning into '698S_BOG'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 25 (delta 7), reused 14 (delta 3), pack-reused 0
Unpacking objects: 100% (25/25), done.
/content/698S_BOG
     |████████████████████████████████| 235kB 4.5MB/s 
     |████████████████████████████████| 102kB 4.9MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 3.1MB 9.6MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 2.7MB 35.0MB/s 
     |████████████████████████████████| 235kB 39.9MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11515 sha256=d86a860b13be10a2f0e342d4fb23f5b56773f71bdd8059a53260975466645905
  Stored 

In [0]:
## 2. Load the list of search sites and search terms

# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd


# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
# 
# Get the website list file and load it to a dataframe
file_id = '1y8FL1rBu8yb2HEn7-ok-ILy1Wq3UrDZU'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('website_list.csv')
website_list_df = pd.read_csv('website_list.csv')
# Get the search term file and load it to a dataframe
file_id = '18MyoEiAYTG6w6JRejv5d4qVK4RxYyRu2'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('search_terms.csv')
search_terms_df = pd.read_csv('search_terms.csv')

#1. Modify the python scripts to target them at the desired websites
Todolist:
 * [ ] edit the python to change the url to a passed variable the targeted site(s) with the desired targets
 * [ ]  

In [0]:
#TODO write a script that removes the generic sites from the imported python code with the desired target(s)
#TODO 

#2. Run the python scriptS

In [44]:
#Option one for calling the file
#pass the list of sites you want to scrape
%cd {repo_dir}
import sixnineeight as scraper
scraper()

/content/698S_BOG


AttributeError: ignored

In [45]:
#Option two for calling the file
%cd /content/698S_BOG
!python3 sixnineeight.py

Streaming output truncated to the last 5000 lines.
2020-06-15 21:18:03 [scrapy.spidermiddlewares.depth] DEBUG: Ignoring link (depth > 1): https://www.cnn.com/2020/06/11/us/tulsa-police-handcuff-teenagers-jaywalking/index.html 
2020-06-15 21:18:03 [scrapy.spidermiddlewares.depth] DEBUG: Ignoring link (depth > 1): https://www.cnn.com/2020/06/11/us/tulsa-police-handcuff-teenagers-jaywalking/index.html 
2020-06-15 21:18:03 [scrapy.spidermiddlewares.depth] DEBUG: Ignoring link (depth > 1): https://www.cnn.com/style/article/banksy-stolen-bataclan-artwork-found-in-italy-trnd/index.html 
2020-06-15 21:18:03 [scrapy.spidermiddlewares.depth] DEBUG: Ignoring link (depth > 1): https://www.cnn.com/style/article/banksy-stolen-bataclan-artwork-found-in-italy-trnd/index.html 
2020-06-15 21:18:03 [scrapy.spidermiddlewares.depth] DEBUG: Ignoring link (depth > 1): https://www.cnn.com/2020/06/11/entertainment/live-pd-ae-police-canceled-trnd/index.html 
2020-06-15 21:18:03 [scrapy.spidermiddlewares.depth] 

#import data analysis libraries

In [0]:
import pandas as pd

In [0]:
pd.read_json('links.json')

,link
0,https://www.cnn.com/
1,https://www.cnn.com/us
2,https://www.cnn.com/world
3,https://www.cnn.com/politics
4,https://www.cnn.com/business
...,...
4344,https://www.cnn.com/health/video/2020/03/27/v8...
4345,https://www.cnn.com/travel/video/2020/03/27/v8...
4346,https://www.cnn.com/interactive/call-to-earth
4347,https://www.cnn.com/specials/world/freedom-pro...
